# Multipool Google Sentiment Analysis

this function will get data from google_result table
and calculate the sentiment analysis. Result is store in google_result_sentiment

crontab command
* * * * * /home/haviz/multipool/ai-case-study/run-ai-sa-google.sh >> /home/haviz/multipool/ai-sa-google.log



In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import pymysql.cursors
import sqlalchemy as sa
import nltk
import string
import os

from nltk.corpus import stopwords 
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import pandas as pd

# nltk.download('stopwords')


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")

In [2]:
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True

def execute_query(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData()

    # Example: Define your table
    your_table = Table('your_table', metadata,
                       Column('id', Integer, primary_key=True),
                       Column('column1', String),
                       Column('column2', String)
                       )

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

In [3]:
import pandas as pd
import re
from sqlalchemy import create_engine

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

def clean_up_tag(comment):
    x_ret = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",comment).split())
    return x_ret

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    
    # remove whitspace
    strOut = strOut.strip()
    
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut

In [4]:
def process_by_df(source_ds, col_name):
    from tqdm import tqdm
    pd.options.mode.chained_assignment = None 
    
    # trim to 2500 character
    source_ds[col_name] = source_ds[col_name].str.slice(0,2000)

    # skip stemming
    source_ds['stemmed'] = source_ds[col_name].apply(clean_up_tag)
    source_ds['stemmed'] = source_ds['stemmed'].apply(casefolding)
    source_ds['stemmed'] = source_ds['stemmed'].apply(text_preproc)

    sw = stopwords.words('indonesian')
    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)
    sw = stopwords.words('indonesian')

    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay

    # Create a new list and insert each element from the original list
    from tqdm import tqdm

    list_text = source_ds['stemmed'].tolist()
    new_list = []
    for i in tqdm(range( len(list_text) )):
        # print(len(list_text[i]))
        try:
            if len(list_text[i]) > 0:
                res = pipe( list_text[i] )
                new_list.append(res)
            else:
                res = pipe( 'kalimat netral' )
                new_list.append(res)
                zip
        except Exception as e:
            # Handle the exception (error) here
            print(f"Error processing value at index {i}: {list_text[i]}")
            print(f"Error message: {str(e)}")
            res = pipe( 'kalimat netral' )
            new_list.append(res)      
            
            continue

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay
    # create a list of our conditions
    for i in range(0, len(new_list)):
        # print(new_list[i][0]['label'])
        if new_list[i][0]['label'] == 'Positive':
            new_list[i][0]['Value'] = 1
        elif new_list[i][0]['label'] == 'Negative':
            new_list[i][0]['Value'] = -1
        elif new_list[i][0]['label'] == 'Neutral':
            new_list[i][0]['Value'] = 0

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']
        source_ds.at[index,'Score'] = new_list[index][0]['score']
        #source_ds.at[index,'Score'] = new_list[index][0]['Value']
        
    return source_ds


In [5]:
# query 400 rows of data query data from table
select_query = "select * \
from news_national_medol gr \
where id not in (select news_national_medol_id from news_national_medol_sentiment) \
order by id desc limit 500"

source_ds = execute_query(select_query)

if len(source_ds) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()
    
# source_ds.head()

In [6]:
ds_title = source_ds
res_ds = process_by_df(ds_title,'title')

  0%|          | 0/23 [00:00<?, ?it/s]

 13%|█▎        | 3/23 [00:00<00:00, 25.42it/s]

 30%|███       | 7/23 [00:00<00:00, 28.22it/s]

 48%|████▊     | 11/23 [00:00<00:00, 29.43it/s]

 61%|██████    | 14/23 [00:00<00:00, 27.83it/s]

 78%|███████▊  | 18/23 [00:00<00:00, 29.18it/s]

 91%|█████████▏| 21/23 [00:00<00:00, 29.32it/s]

100%|██████████| 23/23 [00:00<00:00, 29.03it/s]

In [7]:
res_ds.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,national_medol_id,is_polri,reporter,editor,is_deleted,stemmed,tokenized_tweet,Prediction,Score
0,29568,Arab Saudi Persiapkan Salat Idul Fitri di Dua ...,JEDDAH - Kepresidenan Urusan Agama di Arab Sau...,Rabu 10 April 2024 07:51 WIB,,https://www.okezone.com/,https://news.okezone.com/read/2024/04/10/18/29...,2024-04-10 08:04:08.480148,11,False,,Susi Susanti,False,arab saudi persiapkan salat idul fitri di dua ...,"{fitri, arab, suci, masjidil, persiapkan, sala...",Neutral,0.999746
1,29567,"Idul Fitri 1445 Hijriah, Wapres Ajak Terus Hid...",JAKARTA - Wakil Presiden (Wapres) Ma’ruf Amin ...,Rabu 10 April 2024 07:43 WIB,,https://www.okezone.com/,https://nasional.okezone.com/read/2024/04/10/3...,2024-04-10 08:04:08.477281,11,False,,Binti Mufarida,False,idul fitri hijriah wapres ajak terus hidupkan ...,"{fitri, ramadhan, ajak, hidupkan, wapres, hijr...",Neutral,0.999646
2,29566,Anies Baswedan dan Keluarga Sholat Idul Fitri ...,JAKARTA - Eks Gubernur DKI Jakarta periode (20...,Rabu 10 April 2024 07:32 WIB,,https://www.okezone.com/,https://nasional.okezone.com/read/2024/04/10/3...,2024-04-10 08:04:08.474356,11,False,,Muhammad Refi Sandi,False,anies baswedan dan keluarga sholat idul fitri ...,"{anies, fitri, baswedan, khoirot, h, babul, bu...",Neutral,0.999703
3,29565,Satu Pria Ditangkap Gegara Hendak Masuk Istana...,"JAKARTA - Polisi mengamankan DM, pria berusia ...",Rabu 10 April 2024 07:27 WIB,,https://www.okezone.com/,https://nasional.okezone.com/read/2024/04/10/3...,2024-04-10 08:04:08.471591,11,False,,Jonathan Simanjuntak,False,satu pria ditangkap gegara hendak masuk istana...,"{masuk, kepresidenan, ditangkap, istana, gegar...",Neutral,0.999612
4,29564,Presiden Jokowi dan Ma'ruf Amin Sholat Id di M...,JAKARTA - Presiden Joko Widodo (Jokowi) bersam...,Rabu 10 April 2024 07:27 WIB,,https://www.okezone.com/,https://nasional.okezone.com/read/2024/04/10/3...,2024-04-10 08:04:08.468744,11,False,,Raka Dwi Novianto,False,presiden jokowi dan ma ruf amin sholat id di m...,"{id, istiqlal, jokowi, ruf, ma, sholat, masjid...",Neutral,0.999618


In [8]:
# record result
sentiment_class = 0

# INSERT INTO public.news_national_medol_sentiment (sentiment_category, score, news_national_medol_id) VALUES( 0, 0, 0);

for index, row in res_ds.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "INSERT INTO news_national_medol_sentiment (sentiment_category, score, news_national_medol_id) VALUES(%s, %s, %s);" % (sentiment_class,row['Score'],row['id']) 
    # print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.


In [9]:
ds_content = source_ds
res_ds_content = process_by_df(ds_content,'content')

  0%|          | 0/23 [00:00<?, ?it/s]

  9%|▊         | 2/23 [00:00<00:02,  8.31it/s]

 13%|█▎        | 3/23 [00:00<00:02,  8.08it/s]

 17%|█▋        | 4/23 [00:00<00:02,  8.26it/s]

 22%|██▏       | 5/23 [00:00<00:02,  7.71it/s]

...

 87%|████████▋ | 20/23 [00:02<00:00,  8.34it/s]

 91%|█████████▏| 21/23 [00:02<00:00,  8.58it/s]

 96%|█████████▌| 22/23 [00:02<00:00,  8.76it/s]

100%|██████████| 23/23 [00:02<00:00,  9.02it/s]

100%|██████████| 23/23 [00:02<00:00,  7.78it/s]

In [10]:
res_ds_content.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,national_medol_id,is_polri,reporter,editor,is_deleted,stemmed,tokenized_tweet,Prediction,Score
0,29568,Arab Saudi Persiapkan Salat Idul Fitri di Dua ...,JEDDAH - Kepresidenan Urusan Agama di Arab Sau...,Rabu 10 April 2024 07:51 WIB,,https://www.okezone.com/,https://news.okezone.com/read/2024/04/10/18/29...,2024-04-10 08:04:08.480148,11,False,,Susi Susanti,False,jeddah kepresidenan urusan agama di arab saudi...,"{aspek, dipersiapkan, sepenuhnya, arab, venna,...",Neutral,0.999416
1,29567,"Idul Fitri 1445 Hijriah, Wapres Ajak Terus Hid...",JAKARTA - Wakil Presiden (Wapres) Ma’ruf Amin ...,Rabu 10 April 2024 07:43 WIB,,https://www.okezone.com/,https://nasional.okezone.com/read/2024/04/10/3...,2024-04-10 08:04:08.477281,11,False,,Binti Mufarida,False,jakarta wakil presiden wapres ma ruf amin meng...,"{idulfitri, wal, sanak, irawan, semangat, kasi...",Positive,0.973453
2,29566,Anies Baswedan dan Keluarga Sholat Idul Fitri ...,JAKARTA - Eks Gubernur DKI Jakarta periode (20...,Rabu 10 April 2024 07:32 WIB,,https://www.okezone.com/,https://nasional.okezone.com/read/2024/04/10/3...,2024-04-10 08:04:08.474356,11,False,,Muhammad Refi Sandi,False,jakarta eks gubernur dki jakarta periode anies...,"{mnc, oktober, bulus, irawan, mikail, dki, men...",Neutral,0.999478
3,29565,Satu Pria Ditangkap Gegara Hendak Masuk Istana...,"JAKARTA - Polisi mengamankan DM, pria berusia ...",Rabu 10 April 2024 07:27 WIB,,https://www.okezone.com/,https://nasional.okezone.com/read/2024/04/10/3...,2024-04-10 08:04:08.471591,11,False,,Jonathan Simanjuntak,False,jakarta polisi mengamankan dm pria berusia sek...,"{condro, gangguan, pusat, motif, irawan, istan...",Neutral,0.999556
4,29564,Presiden Jokowi dan Ma'ruf Amin Sholat Id di M...,JAKARTA - Presiden Joko Widodo (Jokowi) bersam...,Rabu 10 April 2024 07:27 WIB,,https://www.okezone.com/,https://nasional.okezone.com/read/2024/04/10/3...,2024-04-10 08:04:08.468744,11,False,,Raka Dwi Novianto,False,jakarta presiden joko widodo jokowi bersama de...,"{pusat, lainnyajakarta, irawan, istana, mengen...",Neutral,0.998952


In [11]:
# record result
sentiment_class = 0

for index, row in res_ds_content.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "update news_national_medol_sentiment set sentiment_category_content = %s, score_content = %s where news_national_medol_id = %s" % (sentiment_class,row['Score'],row['id']) 
    # print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.


...

Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
